In [1]:
# BLOK 1 
# IMPORTOVANIE KNIZNIC

import pandas as pd
import numpy as np
import json
from random import randint


from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, GRU, Concatenate
import tensorflow.keras.utils 
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from ast import literal_eval
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

from tensorflow.keras.models import Sequential

C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nemci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [ ]:
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #
# ----------------------- SYNTHETIC LIGHT CURVES ----------------------- #

In [3]:
# BLOK 2 
# Nacitanie datasetu syntetickych kriviek

data_syn = pd.read_csv(r"C:\Users\nemci\OneDrive\Počítač\Bakalárska práca\Data\df_all.csv_v2")

In [4]:
# BLOK 3
# vyber podmnoziny dotykovych systemov z datasetu syntetickych kriviek

data_over = data_syn[data_syn["morphology"] == "over-contact"]

print("Over-contact light curves: ", data_over.shape[0])

Over-contact light curves:  91189


In [5]:
# BLOK 4
# Diskretizacia 

newDataOver = []
for row in data_over['generic_bessell_v']:
    row = json.loads(row)
    minValueOfRow, maxValueOfRow = min(row), max(row)
    newRow = []
    for valueIndex in range(len(row)):
        row[valueIndex] = (row[valueIndex] - minValueOfRow) / (maxValueOfRow - minValueOfRow)
        newRow.append([row[valueIndex]])
    newDataOver.append(newRow)
newDataOver = np.array(newDataOver)

In [6]:
# BLOK 5
# Vytvorenie cieloveho atributu kriviek. Hodnota 0 pre krivky bez skvrny, hodnota 1 pre skvrnite krivky

target_over = []
over_S, over_No = 0, 0
for row in data_over["spotty"]:
    if row == 1:
        target_over.append(1)
        over_S += 1
    elif row == 0:
        target_over.append(0)
        over_No += 1
target_over = np.array(target_over)
print('Over-contact spotty: ' + str(over_S) + '\nOver-contact no spotty: ' + str(over_No))

Over-contact spotty: 45320
Over-contact no spotty: 45869


In [7]:
# BLOK 6
# Rozdelenie upravenych kriviek na trenovaciu a testovaciu mnozinu v pomere 80:20

print("Number of over-contact light curves: " + str(len(newDataOver)) + "\nTarget attribute: " + str(len(target_over)))
X_trainOver, X_testOver, y_trainOver, y_testOver = train_test_split(newDataOver, target_over, test_size = 0.2)
y_trainOver = to_categorical(y_trainOver, 2)
y_testOver = to_categorical(y_testOver, 2)
print("Length X_trainOver: ", str(len(X_trainOver)))
print("Length y_trainOver: ", str(len(y_trainOver)))
print("Length X_testOver: ", str(len(X_testOver)))
print("Length y_testOver: ", str(len(y_testOver)))

Number of over-contact light curves: 91189
Target attribute: 91189
Length X_trainOver:  72951
Length y_trainOver:  72951
Length X_testOver:  18238
Length y_testOver:  18238


In [ ]:
# BLOK 7
# Vytvorenie architektúry klasifikačneho modelu a jeho trenovanie

model = Sequential()
model.add(Convolution1D(128, 2, activation = 'relu', input_shape = (100, 1))) 
model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(64, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(128, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

savedModel = "new_modelOver.hdf5"
checkpoint = ModelCheckpoint(savedModel, monitor = "val_loss", verbose = 2, save_best_only = True, mode = 'min')
model.fit(X_trainOver, y_trainOver, epochs = 7, batch_size = 32, validation_split=0.2, callbacks = [checkpoint], verbose = 2)

In [8]:
# BLOK 8
# Nacitanie modelu. Predikovanie syntetických dotykovych kriviek a vyhodnotenie modelu na testovacej mnozine syntetických 
# dotykovych kriviek pomocou confusion matrix a vypocet presnosti, navratnosti a f1 skore


model = load_model('new_modelOver.hdf5')
y_pred1DCNN = model.predict(X_testOver)
y_pred2 = []
for i in y_pred1DCNN:
    maximum = np.argmax(i)
    y_pred2 = np.append(y_pred2, maximum)

cm = confusion_matrix(y_testOver.argmax(axis=1), y_pred2)
print("Confusion Matrix 1D CNN (over-contact): \n" + str(cm))

prfs = classification_report(y_testOver.argmax(axis=1), y_pred2)
print(str(prfs))

Confusion Matrix 1D CNN (over-contact): 
[[9099    3]
 [3175 5961]]
              precision    recall  f1-score   support

           0       0.74      1.00      0.85      9102
           1       1.00      0.65      0.79      9136

    accuracy                           0.83     18238
   macro avg       0.87      0.83      0.82     18238
weighted avg       0.87      0.83      0.82     18238

